In [ ]:
import os
import cv2
import numpy as np

In [ ]:
def load_frames_from_folder(folder_path):
    frames = []
    frame_files = sorted(os.listdir(folder_path))  # Sort files to read in correct order
    for frame_file in frame_files:
        if frame_file.endswith(".jpg") or frame_file.endswith(".png"):
            frame = cv2.imread(os.path.join(folder_path, frame_file))
            frames.append(frame)
    return frames

In [ ]:
def duplicate_frames(video_frames, target_num_frames):
    num_frames = len(video_frames)
    if num_frames < target_num_frames:
        # Add more frames by duplicating the last frame
        while len(video_frames) < target_num_frames:
            video_frames.append(video_frames[-1])
    return video_frames

In [ ]:
def interpolate_frames(video_frames, target_num_frames):
    num_frames = len(video_frames)
    if num_frames >= target_num_frames:
        return video_frames
    
    interpolated_frames = []
    for i in range(num_frames - 1):
        start_frame = video_frames[i]
        end_frame = video_frames[i + 1]
        interpolated_frames.append(start_frame)
        # Generate interpolated frames
        for j in range(1, target_num_frames // num_frames):
            alpha = j / (target_num_frames // num_frames)
            interpolated_frame = cv2.addWeighted(start_frame, 1 - alpha, end_frame, alpha, 0)
            interpolated_frames.append(interpolated_frame)
    interpolated_frames.append(video_frames[-1])
    return interpolated_frames

In [ ]:
folders = ["preprocessed_data/Unbalance_weight/front", "preprocessed_data/Unbalance_weight/angle", "preprocessed_data/Bearing_fault/front", "preprocessed_data/Bearing_fault/angle", "preprocessed_data/Normal_state/front", "preprocessed_data/Normal_state/angle"]
all_frames = []

for folder in folders:
    frames = load_frames_from_folder(folder)
    all_frames.append(frames)

# Find the maximum number of frames among all videos
max_frames = max(len(video_frames) for video_frames in all_frames)
print(f"Max number of frames: {max_frames}")

# Apply frame duplication or interpolation to match the max number of frames
all_resized_frames = [duplicate_frames(video_frames, max_frames) for video_frames in all_frames]
# Alternatively, you can use interpolation if you want smoother transitions:
# all_resized_frames = [interpolate_frames(video_frames, max_frames) for video_frames in all_frames]

# Assuming the frames are resized and you want to save them to separate videos
fps = 30  # Choose a common FPS (e.g., 30 FPS)
frame_height, frame_width = all_resized_frames[0][0].shape[:2]  # Assuming all frames are the same size

# List to store the VideoWriter objects for each video
video_writers = []

# Create a VideoWriter object for each folder/video
output_paths = ['merged_preprocessed_videos/Unbalanced_weight/front.mp4', 'merged_preprocessed_videos/Unbalanced_weight/angle.mp4', 'merged_preprocessed_videos/Bearing_fault/front.mp4', 
                'merged_preprocessed_videos/Bearing_fault/angle.mp4', 'merged_preprocessed_videos/Normal_state/front.mp4', 'merged_preprocessed_videos/Normal_state/angle.mp4']

for path in output_paths:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video format
    video_writer = cv2.VideoWriter(path, fourcc, fps, (frame_width, frame_height))
    video_writers.append(video_writer)

# Write frames to each corresponding video
for i, video_frames in enumerate(all_resized_frames):
    for frame in video_frames:
        # Write each frame of the corresponding video to its VideoWriter
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        video_writers[i].write(frame_bgr)

# Release all VideoWriter objects
for writer in video_writers:
    writer.release()

print("Videos saved for each condition.")